# touristAI

This is the second Capstone project as part of the [Artificial Intelligence Nanodegree](https://www.udacity.com/course/artificial-intelligence-nanodegree--nd889), and focuses on Machine Translation Project. 

## Introduction

In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Our completed pipeline will accept English text as input and return the French translation!

## Overview:
- **Preprocess** - We'll convert text to sequence of integers.
- **Models** - We'll create models which accepts a **sequence of integers as input** and returns **a probability distribution over possible translations**. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, and **design our own model**!
- **Prediction** - Lastly, we'll run the model on English text, and read our French!

In [4]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # Using my local CPU; 🙏 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18363151856989339711
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline. 

The most common datasets used for machine translation are from [WMT](http://www.statmt.org/):

![](http://www.statmt.org/book/cover.jpg)

However, that will take a long time to train a neural network on, so we'll be using a sample dataset we created for this project; which contains a much smaller vocabulary. It should be possible to actually train our model in a reasonable time with this dataset. 

### Load Data
The data is located in:
* `data/small_vocab_en`: English sentences.
* `data/small_vocab_fr`. Their French translations.

Let's Load the English and French data from these files from running the cell below.

In [7]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Sample Data

Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  

Let's see what we've got:

In [10]:
for sample_i in range(10):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
small_vocab_en Line 3:  california is usually quiet during march , and it is usually hot in june .
small_vocab_fr Line 3:  california est généralement calme en mars , et il est généralement chaud en juin .
small_vocab_en Line 4:  the united states is sometimes mild during june , and it is cold in september .
small_vocab_fr Line 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
small_vocab_en Line 5:  your least liked fruit is the grape , but my least liked is the apple .
small_vocab_fr Line 5:  votre moins aimé fruit est le raisin , m

From looking at the sentences, you can see they have been preprocessed already:
1. *The puncuations have been delimited using spaces.*
2. *All the text have been converted to lowercase.* 

This should save you some time, (**and are great steps to take in the future**), but the text requires even more preprocessing!

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary; **A more complex vocabulary is a more complex problem.**  

Let's look at the complexity of the dataset we'll be working with:

In [13]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print('\n')
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"


1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words. 

We have therefore traded vocabulary *depth* for optimal translation accuracy, supporting our smaller dictionary with *significantly more examples*.

## Preprocess
For this project, **we won't use text data as input to your model, instead we'll encode our text into sequences of integers**, using the following preprocess methods:

3. *Tokenizing the words into ids.*
4. *Adding padding to make all the sequences the same length.*

Let's go!

### Tokenize 
For a neural network **to predict on text data**, it first **has to be turned into data it can understand.**

Text data like "dog" is actually sequence of ASCII character encodings, stored and encoded in memory: `100 111 103`. 

![](https://www.w3.org/International/articles/definitions-characters/index-data/encodings.png)

Therefore, texts are just a specific format for storage, and can be converted to and from, sort of like a machine translation *inside* our language translator! 

![](https://imgflip.com/s/meme/Yo-Dawg-Heard-You.jpg)

Let's convert *these* numbers into arrays we can run a series of multiplication and addition operations on, and then convert the *output* back into ASCII encoding!

To do this, we need to give each character or each word into a unique ID. These are called **character or word ids**, respectively:
* **Character IDs**: We'd use character ids for *character level models* that generate text predictions for *each character*. 
* **Word IDs**: A *word level model* uses *word ids* that generate text predictions for each word, so we'd need to encode each word as a unique value.

Word level models tend to learn better, since they are lower in complexity, so we'll use one of those.

First, let's turn each sentence into a sequence of words ids using Keras's (built-in!) [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Here's that function running on our `english_sentences` and `french_sentences`:

In [29]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Initialize 
    tokenizer = Tokenizer(num_words=None, 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
                     lower=True, 
                     split=" ", 
                     char_level=False)
    
    # Fit
    tokenizer.fit_on_texts(x)
    
    # Take sequences
    tokenized = tokenizer.texts_to_sequences(x)
    
    return tokenized, tokenizer

    return None, None
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding 

When batching the sequence of word ids together, each sequence needs to be the same length, otherwise our linear algebra will trip over misshapen matrices. This is a problem, since **sentences are dynamic in length**, but we can solve it by **adding padding to the end of the sequences to make them the same length**.

As long as we make sure all the English sequences have the same length *and* all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's (built in!) [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function. Let's see how it works: 

In [32]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    
    ## Pad sequences to given length, or use the longest subsequence
    length = length if length else max([len(seequence) for seequence in x])
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

Our focus for this project is to build neural network architecture, so we won't create a full preprocess pipeline.  

Instead, we've provided you with the implementation of the `preprocess` function:

In [34]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    # Tokenize!
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    # Pad!
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models

In this section, we will experiment with various neural network architectures, and see how they do at our problem: **translating English to French using a word-level dictioanry.**

To do this, we will begin by training four relatively simple architectures.
1. **Model 1** is a *basic RNN*
2. **Model 2** is a *RNN with Embedding*
3. **Model 3** is a *Bidirectional RNN*
4. **Model 4** is an *Optional Encoder-Decoder RNN*

After experimenting with the four simple architectures, **we will construct a deeper architecture that is designed to outperform all four models.**

### IDs Back to Text

The neural network will be translating the input to words ids, which isn't the final form we want; we want to **use our French dcitionary to convert word IDs back to French.**

The function `logits_to_text` will bridge the gab between the logits (local units) from the neural network to the French translation. We'll also be using this function to better understand the output of the neural network!

In [35]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Hyperparameters

Before we get started, let's define some overall hyperparameters for all of our models, for fair comparison:

In [54]:
# Hyperparameters
learning_rate = 0.05   # 5% Learning rate
batch_size = 1024      # Uniform batch size 
epochs = 5             # An epoch is 5 batches 
validation_split = 0.2 # Leave 20% for Validation

### Model 1: Basic RNN 

A basic RNN model is generally good baseline for sequence data.  In this model, we'll build this base RNN that translates English to French:

![RNN](images/rnn.png)

In [53]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    ## Basic Recurrent Neural Network that:
    # * Accepts padded, reshaped uniform input and output sequences.
    # * Use a connected recurrent neural network to map word IDs

    model = Sequential()                                # Linear Layer Stack
    
    model.add(GRU(input_shape=input_shape[1:], 
                        units=750,
                        return_sequences=True)) # Input Layer: Simple RNN
                                                # Returns in sequences (Sentences vs words)
                                                # Inherent 20% dropout 
            
    model.add(TimeDistributed(
        Dense(units=2*french_vocab_size, 
              activation='relu')))              # Layer 2: Time Distributed Dense Layer
                                                # Allows every timestep to be directly connected to
                                                # Scale size for more connections
                                                # Rectified Linear Activation Function
    
    
    model.add(TimeDistributed(
        Dense(units=french_vocab_size, 
              activation='softmax')))           # Output Layer: Time Distributed Dense Layer
                                                # Shrink back down to required dictionary
                                                # Softmax for final acitivation / word translation
            
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])         # Compile 
                                                # Loss via Categorical Cross Entropy
                                                # Optimize via Adam
                                                # Accuracy KPI
    
    return model

### Test
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN

console.log(preproc_english_sentences)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
console.log(tmp_x)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
console.log(tmp_x)
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

NameError: name 'learning_rate' is not defined

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    return None
tests.test_embed_model(embed_model)


# TODO: Reshape the input

# TODO: Train the neural network

# TODO: Print prediction(s)

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    return None
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    return None
tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?